In [ ]:
import os
import glob
from PIL import Image, ImageFilter
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
from tqdm import tqdm

In [ ]:
import os
from PIL import Image

def bicubic_downsample(input_path, output_path, scale=2):
    """
    Downsamples an image using bicubic interpolation.

    Args:
        input_path (str): Path to the input high-resolution image.
        output_path (str): Path to save the downsampled image.
        scale (int): Downsampling factor (e.g., 2 means image size will be halved).
    """
    # Open image
    img = Image.open(input_path).convert("RGB")
    w, h = img.size

    # Compute new size
    new_w = w // scale
    new_h = h // scale

    # Bicubic downsampling
    img_down = img.resize((new_w, new_h), Image.BICUBIC)

    # Ensure output directory exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Save result
    img_down.save(output_path)

    print(f"✅ Downsampled image saved at: {output_path}")
    print(f"Original size: {w}x{h}, Downsampled size: {new_w}x{new_h}")

# -------------------------------
# Example usage
# -------------------------------
if __name__ == "__main__":
    input_image = "/content/richard-avedon-marilyn-monroe-min-1.jpg"      # path to original HR image
    output_image = "downsampled/sample_x2.jpg"   # save path
    scale_factor = 2                             # e.g., 2x downsample

    bicubic_downsample(input_image, output_image, scale=scale_factor)


✅ Downsampled image saved at: downsampled/sample_x2.jpg
Original size: 604x537, Downsampled size: 302x268


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F # Import F for relu and interpolate
from torchvision import transforms, models # Import models for VGG
from PIL import Image, ImageFilter # Import ImageFilter for potential post-processing
import os
import glob # Import glob for finding files if needed later

# ====================================
# CONFIGURATION
# ====================================
MODEL_PATH = "/content/tiny_espcn_celeba.pth"   # Path to your trained model
INPUT_IMAGE = "/content/downsampled/sample_x2.jpg"              # Low-resolution input image
OUTPUT_IMAGE = "/content/sample_sr_2.png"          # Super-resolved output path
UPSCALE_FACTOR = 2                              # Set same as training
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ====================================
# MODEL DEFINITION (TinyESPCN Enhanced)
# ====================================
# Channel Attention module (copied from training code)
class ChannelAttention(nn.Module):
    def __init__(self, channels, reduction=16):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels//reduction, bias=False),
            nn.ReLU(),
            nn.Linear(channels//reduction, channels, bias=False),
            nn.Sigmoid()
        )
    def forward(self, x):
        b,c,_,_ = x.size()
        y = self.avg_pool(x).view(b,c)
        y = self.fc(y).view(b,c,1,1)
        return x * y

# TinyESPCN Enhanced Model (copied from training code)
class TinyESPCNEnhanced(nn.Module):
    def __init__(self, scale=2, use_attention=True):
        super().__init__()
        self.scale = scale
        self.use_attention = use_attention

        self.conv1 = nn.Conv2d(3,64,7,1,3)
        self.res_blocks = nn.Sequential(*[nn.Sequential(nn.Conv2d(64,64,3,1,1), nn.ReLU()) for _ in range(10)])
        if use_attention:
            self.attention = ChannelAttention(64)
        self.conv2 = nn.Conv2d(64, 3*(scale**2), 3,1,1)
        self.pixel_shuffle = nn.PixelShuffle(scale)

    def forward(self, x):
        lr_input = x
        x1 = F.relu(self.conv1(x))
        x2 = self.res_blocks(x1)
        if self.use_attention:
            x2 = self.attention(x2)
        x = self.pixel_shuffle(self.conv2(x2+x1))
        lr_up = F.interpolate(lr_input, scale_factor=self.scale, mode='bicubic', align_corners=False)
        return torch.clamp(x+lr_up,0,1)


# ====================================
# LOAD MODEL
# ====================================
model = TinyESPCNEnhanced(scale=UPSCALE_FACTOR) # Use the Enhanced model definition
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval()

# ====================================
# TRANSFORMS
# ====================================
transform = transforms.Compose([
    transforms.ToTensor()
])
to_pil = transforms.ToPILImage()

# ====================================
# LOAD INPUT IMAGE
# ====================================
image = Image.open(INPUT_IMAGE).convert("RGB")
input_tensor = transform(image).unsqueeze(0).to(DEVICE)

# ====================================
# INFERENCE
# ====================================
with torch.no_grad():
    output_tensor = model(input_tensor)

output_tensor = torch.clamp(output_tensor.squeeze(0), 0, 1)
output_image = to_pil(output_tensor.cpu())

# ====================================
# SAVE OUTPUT IMAGE
# ====================================
os.makedirs(os.path.dirname(OUTPUT_IMAGE), exist_ok=True)
output_image.save(OUTPUT_IMAGE)
print(f"✅ Super-resolution complete! Saved at: {OUTPUT_IMAGE}")

#Optional display
output_image.show()

✅ Super-resolution complete! Saved at: /content/sample_sr_2.png
